In [12]:
import pandas as pd
import pm_lib
import ipywidgets as widgets
from IPython.display import clear_output, display
from tkinter import Tk, filedialog

In [68]:
files = ''

def select_files(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True) # Raise the root to the top of all windows.
    files = filedialog.askopenfilename(title='Choose a File', filetypes=[("JFLAP files", "*.jff")]) 

def select_files_multiple(b):
    global files
    clear_output()
    root = Tk()
    root.withdraw() 
    root.call('wm', 'attributes', '.', '-topmost', True) 
    files = filedialog.askopenfilename(title='Choose a File', multiple=True, filetypes=[("JFLAP files", "*.jff")]) 

layout = widgets.Layout(width='170px')

fileselect = widgets.Button(description="Selecione o arquivo", layout = layout)
fileselect.on_click(select_files)

fileselect_multiple = widgets.Button(description="Selecione os arquivos", layout = layout)
fileselect_multiple.on_click(select_files_multiple)


In [69]:
col_names = ['Word', 'Expected']


def on_button_clicked_nfa(b):
    global files
    results = []
    
    with out1:
        nfa = pm_lib.jffToNFA(files)
        doc = 'nfa.cases'
        g = pm_lib.nfa_to_graph(nfa)
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        results = pm_lib.nfa_validate(nfa, doc)
        df = pd.DataFrame(columns=["Received"])
        df["Received"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        df.to_csv('file_sent.csv', sep='\t', encoding='utf-8', index=False)
        clear_output()
        display(g)
        display(df)

def on_button_clicked_dfa(b):
    global files
    results = []
    
    with out2:
        dfa = pm_lib.jffToDFA(files)
        doc = 'dfa.cases'
        g = pm_lib.dfa_to_graph(dfa)
        data = pd.read_csv(doc, sep='\t', dtype=object, names = col_names)
        results = pm_lib.dfa_validate(dfa, doc)
        df = pd.DataFrame(columns=["Received"])
        df["Received"] = results
        df = pd.concat([data, df], axis=1)
        df = df.dropna()
        df.to_csv('file_sent.csv', sep='\t', encoding='utf-8', index=False)
        clear_output()
        display(g)
        display(df)



In [70]:
def on_button_clicked_union(b):
    global files

    with out3:
        automata1 = pm_lib.jffToNFA(files[0])
        automata2 = pm_lib.jffToNFA(files[1])
        united_automata = pm_lib.automata_nfa_union(automata1, automata2)
        g = pm_lib.nfa_to_graph(united_automata)
        clear_output()
        display(g)

def on_button_clicked_intersection(b):
    global files

    with out4:
        automata1 = pm_lib.jffToNFA(files[0])
        automata2 = pm_lib.jffToNFA(files[1])
        intersection_automata = pm_lib.automata_nfa_intersection(automata1, automata2)
        g = pm_lib.nfa_to_graph(intersection_automata)
        clear_output()
        display(g)


In [71]:
# file = open("images/ufc-brasao.png", "rb")
# image = file.read()
# widgets.Image(
#     value=image,
#     format='png',
#     width=300,
#     height=400,
# )

In [105]:
out1 = widgets.Output()
out2 = widgets.Output()
out3 = widgets.Output()
out4 = widgets.Output()
out5 = widgets.Output()

button_nfa = widgets.Button(description='Avaliar', layout = layout)
button_dfa = widgets.Button(description='Avaliar', layout = layout)
button_union = widgets.Button(description='União', layout = layout)
button_intersection = widgets.Button(description='Interseção', layout = layout)

button_nfa.on_click(on_button_clicked_nfa)
button_dfa.on_click(on_button_clicked_dfa)
button_union.on_click(on_button_clicked_union)
button_intersection.on_click(on_button_clicked_intersection)

nfa_title =  widgets.HTML(value="<h3>Autômato Finito não Determinístico</h3>")
nfa_description = widgets.HTML(value= "<p> Um autômato finito não determinístico" +
    " pode ter transições para mais de um estado ao ler como entrada " +
    "um mesmo valor. </p>" +
    "<p> Clique em avaliar para ver o resultado do NFA em relação aos valores esperados. </p>"
    )

dfa_title =  widgets.HTML(value="<h3>Autômato Finito Determinístico</h3>")
dfa_description = widgets.HTML(value="<p> Um autômato finito determinístico" +
    " não pode ter transições para mais de um estado ao ler como entrada " +
    "um mesmo valor. </p>"
    "<p> Clique em avaliar para ver o resultado do DFA em relação aos valores esperados. </p>"
    )

union_title =  widgets.HTML(value="<h3>União</h3>")
union_decription = widgets.HTML(value="<img src='images/union.png' alt='União' style='width:220px;height:132px;'>"+
    " <p> Você pode realizar a união de dois autômatos. "+
    "Selecione os dois arquivos '.jff' que deseja unir e clique em 'União'. </p>"
    )

intersection_title =  widgets.HTML(value="<h3>Interseção</h3>")
intersection_decription = widgets.HTML(value="<img src='images/intersection.png' alt='Interseção'>"+
    " <p> Você pode realizar a interseção de dois autômatos. "+
    "Selecione os dois arquivos '.jff' que deseja fazer a interseção e clique em 'Interseção'. </p>"
    )
    
vbox1 = widgets.VBox([nfa_title, nfa_description, fileselect, out1, button_nfa])
vbox2 = widgets.VBox([dfa_title, dfa_description, fileselect, out2, button_dfa])
vbox3 = widgets.VBox([union_title, union_decription, fileselect_multiple, out3, button_union])
vbox4 = widgets.VBox([intersection_title, intersection_decription, fileselect_multiple, out4, button_intersection])
# vbox_about = widgets.VBox([image, acord1, acord2])

tab = widgets.Tab(children = [vbox1, vbox2, vbox3, vbox4])
tab.set_title(0, 'NFA')
tab.set_title(1, 'DFA')
tab.set_title(2, 'União')
tab.set_title(3, 'Interseção')

display(tab)



In [107]:
# Um DFA pode ser definido pela quíntupla:

# \begin{center}
#     $M = (Q, \Sigma, \delta, q_0, F)$
# \end{center}
# onde:
# \begin{itemize}
#     \item Q é o conjunto dos estados;
#     \item $\Sigma$ é o alfabeto de entrada;
#     \item $\delta$ é a função de transição onde: $Q \times \Sigma \rightarrow Q$;
#     \item $q_0$ é o estado inicial;
#     \item F é o conjunto de estados finais.
# \end{itemize}
# A utilização do termo \textit{determinístico} se dá pelo motivo de que enquanto houver um símbolo de entrada a ser lido sempre será possível determinar o próximo estado que o autômato irá assumir e esse estado será único em todas as situações. 


# Para representar um DFA de maneira gráfica, podemos usar uma \textbf{diagrama de transições} ou uma \textbf{tabela de transições}. Um diagrama de transições é definido da forma que se segue:

# \begin{itemize}
#     \item Para cada estado $q \in Q$, existe um nó \textit{q} correspondente;
#     \item Para cada estado $q \in Q$ e para cada símbolo da entrada $a \in \Sigma$ tal que $\delta (q, a) = p$, o diagrama tem um arco que sai do nó \textit{q} e vai para o nó \textit{p} rotulado por \textit{a};
#     \item Existe uma seta apontando para o $q_0$ indicando-o como estado inicial;
#     \item Os nós que representam os estados finais possuem um círculo duplo, diferentemente dos demais nós que possuem apenas um.
# \end{itemize}


# Um exemplo de DFA que tem como alfabeto $\Sigma = {0, 1}$ e aceita palavras começadas por 01 é mostrado na Figura \ref{fig:dfa}.


# NFAs podem ser vistos como uma espécie de autômatos onde é realizado uma computação paralela onde múltiplos processos são realizados concorrentemente. O autômato finito não determinístico se divide para seguir a diversas escolhas de caminhos que ocorrerão paralelamente, logo é gerada uma árvore de computação onde, se pelo menos um caminho chegar em um estado final, então toda a computação será aceita.


# Uma variação do NFA permite que sejam realizadas transições sobre o caractere vazio, representado pela letra $\epsilon$. Um autômato finito com transições vazias, ou $\epsilon$-NFA, é definido da forma $A_{\epsilon} = (Q, \Sigma, \delta_{\epsilon}, q_0, F)$, onde:


# \begin{itemize}
#     \item Q é o conjunto de estados;
#     \item $\Sigma$ é o alfabeto de entrada;
#     \item $q_0$ é o estado inicial;
#     \item F é o conjunto de estados finais;
#     \item $\delta_{\epsilon}: Q \times \Sigma \cup \{ \epsilon \} \rightarrow 2^Q$.
# \end{itemize}
# Para entendermos como é feita a computação de um $\epsilon$-NFA é necessário nos atentarmos para o conceito de $\epsilon$-fechamento. O $\epsilon$-fechamento de um nó q, representado por $S_{\epsilon}(q)$ diz que:

# \begin{itemize}
#     \item $q \in S_{\epsilon}(q)$;
#     \item Se $p \in S_{\epsilon}(q)$ e existe uma transição vazia a partir de p, então $\delta (p, \epsilon) \subseteq S_{\epsilon}(q)$.
# \end{itemize}


# Assim, para definir uma computação, temos que a função de transição estendida, representada por $\overline{\delta}{\epsilon}$, se dá da forma: $\overline{\delta}{\epsilon}: Q \times \Sigma^* \rightarrow 2^Q$. Formalmente, temos:

# \begin{center}
#     $\overline{\delta}{\epsilon}(q, \epsilon) = S{\epsilon}(q)$
    
#     $\overline{\delta}{\epsilon}(q, xa) = \bigcup{r \in R} S_{\epsilon}(r)$, onde $R = \bigcup_{p \in \overline{\delta}{\epsilon}(p, x)}^{} \delta{\epsilon}(p, a)$
# \end{center}